In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
# import pickle
import time
# from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb
# import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# csv融合

# zero特征

In [4]:
train = pd.read_csv('train_zero_model2.csv')

train.shape
train.head()

(2877748, 174)

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,minute,user_query_day,user_query_day_hour,user_query_minute,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_hot,user_hot,shop_hot,brand_hot,occupation_hot,city_hot,item_trade_hot,user_trade_hot,shop_trade_hot,brand_trade_hot,occupation_trade_hot,city_trade_hot,user_sell_power,user_sell_power_mean,user_sell_power_mean_item_minus,user_sell_trade_power,user_sell_trade_power_mean,user_sell_trade_power_mean_item_minus,yesterday,yesterhour,user_day_query,user_yesterday_query,user_hour_query,user_yesterhour_query,user_minute_query,item_day_query,item_hour_query,item_minute_query,user_item_day_query,user_item_hour_query,user_item_minute_query,min_time,user_is_his,item_is_his,brand_is_his,user_is_trade_his,item_is_trade_his,brand_is_trade_his,item_trade_percent,brand_trade_percent,user_trade_percent,occupation_trade_percent,is_home,shop_item_count,brand_item_count,city_brand_count,city_item_count,shop_item_count_his,brand_item_count_his,city_brand_count_his,city_item_count_his,user_count,user_day_count,user_hour_count,user_minute_count,user_next_count,user_last_count,user_next_time,user_last_time,next_time_sub,last_time_sub,user_item_num,user_shop_num,user_item_day_num,user_shop_day_num,user_item_num_next,user_item_num_last,user_shop_num_next,user_shop_num_last,user_item_next_time,user_item_last_time,user_shop_next_time,user_shop_last_time,user_item_next_time_sub,user_item_last_time_sub,user_shop_next_time_sub,user_shop_last_time_sub,user_fisrt_num,user_item_fisrt_num,user_shop_fisrt_num,user_count_first_time,user_item_count_first_time,user_shop_count_first_time,user_count_first_time_sub,user_item_count_first_time_sub,user_shop_count_first_time_sub,max_click,is_last_click,max_user_item_click,is_last_user_item_click,max_click_time,max_user_item_click_time,max_click_time_sub,max_user_item_click_time_sub,instance_id_full_time,next_time_sub_full_time,last_time_sub_full_time,user_shop_num_full_time,user_item_day_num_full_time,user_shop_day_num_full_time,user_item_next_time_sub_full_time,user_item_last_time_sub_full_time,user_shop_next_time_sub_full_time,user_shop_last_time_sub_full_time,user_count_first_time_sub_full_time,user_item_count_first_time_sub_full_time,user_shop_count_first_time_sub_full_time,max_click_full_time,is_last_click_full_time,max_user_item_click_full_time,is_last_user_item_click_full_time,max_click_time_sub_full_time,max_user_item_click_time_sub_full_time
0,3301974838081842019,3044249002439294401,8408375359349630601,3122721854741763495,6,10,12,16,1748764844194575251,0,1004,2005,3002,5639892653771508251,1535990400,10,3697448581765982326,18,1.0,16,0.974200,0.974506,0.980671,0,2018-09-04 00:00:00,4,0,5,5,5,5,2,25,1,1,1,0,3.333333,0.000000,5.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,59.0,111.0,0.0,0.0,0.0,31.654479,202,25.250000,2.104167,139.0,0.0,1996.0,1996.0,2997852,93033.0,0.0,0.0,7.0,7.0,40047,1257.0,0.0,0.00,6.00,0.0,0.0,6.0,3,23,5,0.0,5,0.0,5,39,3,2,1,1,1,0.0,1,0,1,1,1,0,0.000000,0.350701,0.0,1.335856,0,13,13,167,870,16

In [5]:
test = train[train.is_trade == -1]
train = train[train.is_trade != -1]

test.shape
train.shape

(519888, 174)

(2357860, 174)

In [6]:
origin = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id',
    'user_star_level', 'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

features = [
    'day', 'hour', 'minute', 'user_query_day', 'user_query_day_hour',
    'user_query_minute', 'item_category_split_count',
    'item_property_split_count', 'pred_category_split_count',
    'pred_property_split_count', 'item_pred_category_score',
    'item_pred_property_score', 'item_pred_category_score_item%',
    'item_pred_property_score_item%', 'item_pred_category_score_pred%',
    'item_pred_property_score_pred%', 'user_shop_count', 'user_item_count',
    'user_context_count', 'user_shop_trade_count', 'user_item_trade_count',
    'user_context_trade_count', 'user_brand_count', 'user_city_count',
    'user_brand_trade_count', 'user_city_trade_count', 'item_occupation_count',
    'item_age_count', 'item_gender_count', 'item_occupation_trade_count',
    'item_age_trade_count', 'item_gender_trade_count', 'shop_score',
    'item_score', 'item_score2', 'item_score3', 'item_hot', 'user_hot',
    'shop_hot', 'brand_hot', 'occupation_hot', 'city_hot', 'item_trade_hot',
    'user_trade_hot', 'shop_trade_hot', 'brand_trade_hot',
    'occupation_trade_hot', 'city_trade_hot', 'user_sell_power',
    'user_sell_power_mean', 'user_sell_power_mean_item_minus',
    'user_sell_trade_power', 'user_sell_trade_power_mean',
    'user_sell_trade_power_mean_item_minus', 'yesterday', 'yesterhour',
    'user_day_query', 'user_yesterday_query', 'user_hour_query',
    'user_yesterhour_query', 'user_minute_query', 'item_day_query',
    'item_hour_query', 'item_minute_query', 'user_item_day_query',
    'user_item_hour_query', 'user_item_minute_query', 'min_time',
    'user_is_his', 'item_is_his', 'brand_is_his', 'user_is_trade_his',
    'item_is_trade_his', 'brand_is_trade_his', 'user_count', 'user_day_count',
    'user_hour_count', 'user_minute_count', 'item_trade_percent',
    'brand_trade_percent', 'user_trade_percent', 'occupation_trade_percent',
    'is_home', 'shop_item_count', 'brand_item_count', 'city_brand_count',
    'city_item_count', 'shop_item_count_his', 'brand_item_count_his',
    'city_brand_count_his', 'city_item_count_his', 'next_time_sub',
    'last_time_sub', 'user_item_num', 'user_shop_num', 'user_item_day_num',
    'user_shop_day_num', 'user_item_next_time_sub', 'user_item_last_time_sub',
    'user_shop_next_time_sub', 'user_shop_last_time_sub',
    'user_count_first_time_sub', 'user_item_count_first_time_sub',
    'user_shop_count_first_time_sub', 'max_click', 'is_last_click',
    'max_user_item_click', 'is_last_user_item_click', 'max_click_time_sub',
    'max_user_item_click_time_sub'
]

features_fulltime = [
    'next_time_sub_full_time', 'last_time_sub_full_time',
    'user_shop_num_full_time',
    'user_item_day_num_full_time', 'user_shop_day_num_full_time',
    'user_item_next_time_sub_full_time', 'user_item_last_time_sub_full_time',
    'user_shop_next_time_sub_full_time', 'user_shop_last_time_sub_full_time',
    'user_count_first_time_sub_full_time',
    'user_item_count_first_time_sub_full_time',
    'user_shop_count_first_time_sub_full_time', 'max_click_full_time',
    'is_last_click_full_time', 'max_user_item_click_full_time',
    'is_last_user_item_click_full_time', 'max_user_item_click_time_sub_full_time'
]

target = ['is_trade']

In [7]:
predictors = features + features_fulltime
len(origin)
len(predictors)
len(set(predictors) & set(train.columns))

15

127

127

In [14]:
#baseline
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

tmp_predictors = origin.copy()
tmp_predictors = origin + good_features
len(tmp_predictors)
xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=4000,
#     verbose_eval=100,
    nfold=3,
    early_stopping_rounds=50,
    seed=201803)
    
print('Baseline->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))
    
# Baseline->400:0.0535996666667

111

Baseline->1133:0.0502233333333


In [9]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

feature_peoformance = {}

for i in enumerate(predictors):
    if i[1] not in feature_peoformance.keys():
        tmp_predictors = origin.copy()
        tmp_predictors.append(i[1])
        xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

        cv = xgb.cv(
            params,
            dtrain=xgbtrain,
            num_boost_round=400,
            nfold=3,
            early_stopping_rounds=50,
            seed=201803)
    
        feature_peoformance[i[1]] = cv.iloc[-1, :]['test-logloss-mean']
        print(str(i[0]) + ':' + str(i[1]) + '->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))
        
# Baseline->400:0.0535996666667(粗筛)
# 1133:0.0502233333333(96)

0:day->400:0.0536016666667
1:hour->400:0.0534356666667
2:minute->400:0.05361
3:user_query_day->400:0.0533146666667
4:user_query_day_hour->400:0.053278
5:user_query_minute->400:0.053143
6:item_category_split_count->400:0.0535996666667
7:item_property_split_count->400:0.053559
8:pred_category_split_count->400:0.05361
9:pred_property_split_count->400:0.053606
10:item_pred_category_score->400:0.0536
11:item_pred_property_score->400:0.0536056666667
12:item_pred_category_score_item%->400:0.053581
13:item_pred_property_score_item%->400:0.053601
14:item_pred_category_score_pred%->400:0.0536013333333
15:item_pred_property_score_pred%->400:0.0536006666667
16:user_shop_count->400:0.053591
17:user_item_count->400:0.0535933333333
18:user_context_count->400:0.053606
19:user_shop_trade_count->400:0.0536126666667
20:user_item_trade_count->400:0.0536053333333
21:user_context_trade_count->400:0.053606
22:user_brand_count->400:0.0536103333333
23:user_city_count->400:0.0536033333333
24:user_brand_trade_co

In [10]:
len(feature_peoformance)
feature_peoformance

127

{'brand_hot': 0.053520666666666668,
 'brand_is_his': 0.053605999999999994,
 'brand_is_trade_his': 0.053602000000000004,
 'brand_item_count': 0.053545000000000002,
 'brand_item_count_his': 0.053533999999999998,
 'brand_trade_hot': 0.053498666666666667,
 'brand_trade_percent': 0.05314633333333333,
 'city_brand_count': 0.053543333333333332,
 'city_brand_count_his': 0.053546333333333335,
 'city_hot': 0.053549333333333331,
 'city_item_count': 0.053548999999999992,
 'city_item_count_his': 0.053543666666666663,
 'city_trade_hot': 0.053535666666666669,
 'day': 0.053601666666666659,
 'hour': 0.053435666666666666,
 'is_home': 0.053603666666666661,
 'is_last_click': 0.052832666666666667,
 'is_last_click_full_time': 0.052832666666666667,
 'is_last_user_item_click': 0.053353000000000005,
 'is_last_user_item_click_full_time': 0.053353000000000005,
 'item_age_count': 0.053613999999999995,
 'item_age_trade_count': 0.053398333333333332,
 'item_category_split_count': 0.053599666666666664,
 'item_day_que

In [11]:
baseline = 0.0535996666667
good_features = []

for i in feature_peoformance.keys():
    if feature_peoformance[i] < baseline:
        good_features.append(i)
len(good_features)

96

In [12]:
good_features

['hour',
 'user_query_day',
 'user_query_day_hour',
 'user_query_minute',
 'item_category_split_count',
 'item_property_split_count',
 'item_pred_category_score_item%',
 'user_shop_count',
 'user_item_count',
 'user_city_trade_count',
 'item_occupation_trade_count',
 'item_age_trade_count',
 'item_gender_trade_count',
 'shop_score',
 'item_score2',
 'item_hot',
 'user_hot',
 'shop_hot',
 'brand_hot',
 'city_hot',
 'item_trade_hot',
 'user_trade_hot',
 'shop_trade_hot',
 'brand_trade_hot',
 'city_trade_hot',
 'user_sell_power',
 'user_sell_power_mean',
 'user_sell_power_mean_item_minus',
 'user_sell_trade_power',
 'user_sell_trade_power_mean',
 'user_sell_trade_power_mean_item_minus',
 'yesterhour',
 'user_day_query',
 'user_yesterday_query',
 'user_hour_query',
 'user_minute_query',
 'item_day_query',
 'item_hour_query',
 'item_minute_query',
 'user_item_day_query',
 'user_item_hour_query',
 'min_time',
 'user_is_his',
 'user_is_trade_his',
 'item_is_trade_his',
 'user_count',
 'user_d

In [ ]:
['hour',
 'user_query_day',
 'user_query_day_hour',
 'user_query_minute',
 'item_category_split_count',
 'item_property_split_count',
 'item_pred_category_score_item%',
 'user_shop_count',
 'user_item_count',
 'user_city_trade_count',
 'item_occupation_trade_count',
 'item_age_trade_count',
 'item_gender_trade_count',
 'shop_score',
 'item_score2',
 'item_hot',
 'user_hot',
 'shop_hot',
 'brand_hot',
 'city_hot',
 'item_trade_hot',
 'user_trade_hot',
 'shop_trade_hot',
 'brand_trade_hot',
 'city_trade_hot',
 'user_sell_power',
 'user_sell_power_mean',
 'user_sell_power_mean_item_minus',
 'user_sell_trade_power',
 'user_sell_trade_power_mean',
 'user_sell_trade_power_mean_item_minus',
 'yesterhour',
 'user_day_query',
 'user_yesterday_query',
 'user_hour_query',
 'user_minute_query',
 'item_day_query',
 'item_hour_query',
 'item_minute_query',
 'user_item_day_query',
 'user_item_hour_query',
 'min_time',
 'user_is_his',
 'user_is_trade_his',
 'item_is_trade_his',
 'user_count',
 'user_day_count',
 'user_hour_count',
 'user_minute_count',
 'item_trade_percent',
 'brand_trade_percent',
 'user_trade_percent',
 'shop_item_count',
 'brand_item_count',
 'city_brand_count',
 'city_item_count',
 'shop_item_count_his',
 'brand_item_count_his',
 'city_brand_count_his',
 'city_item_count_his',
 'next_time_sub',
 'last_time_sub',
 'user_item_num',
 'user_shop_num',
 'user_item_day_num',
 'user_shop_day_num',
 'user_item_next_time_sub',
 'user_item_last_time_sub',
 'user_shop_next_time_sub',
 'user_shop_last_time_sub',
 'user_count_first_time_sub',
 'user_item_count_first_time_sub',
 'user_shop_count_first_time_sub',
 'max_click',
 'is_last_click',
 'max_user_item_click',
 'is_last_user_item_click',
 'max_click_time_sub',
 'max_user_item_click_time_sub',
 'next_time_sub_full_time',
 'last_time_sub_full_time',
 'user_shop_num_full_time',
 'user_item_day_num_full_time',
 'user_shop_day_num_full_time',
 'user_item_next_time_sub_full_time',
 'user_item_last_time_sub_full_time',
 'user_shop_next_time_sub_full_time',
 'user_shop_last_time_sub_full_time',
 'user_count_first_time_sub_full_time',
 'user_item_count_first_time_sub_full_time',
 'user_shop_count_first_time_sub_full_time',
 'max_click_full_time',
 'is_last_click_full_time',
 'max_user_item_click_full_time',
 'is_last_user_item_click_full_time',
 'max_user_item_click_time_sub_full_time']

# CJF特征

In [3]:
import gc

In [18]:
train = pd.read_csv('feature_full_v1_457_new.csv', nrows=1)

predictors = list(set(train.columns) - set(origin) - set(['day']))
len(predictors)

889

In [14]:
train = pd.read_csv('feature_full_v1_457_new.csv', usecols=list(origin + ['is_trade', 'day']))

train.shape

(3954923, 17)

In [16]:
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

train = train[train.day != 7]
tmp_predictors = origin.copy()
xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

cv = xgb.cv(
        params,
        dtrain=xgbtrain,
        num_boost_round=400,
        nfold=3,
        early_stopping_rounds=50,
        seed=201803)
print('Baseline->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))

Baseline->400:0.0536056666667


In [ ]:
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

feature_performance = {}

for k in range(8, 10):
    train = pd.read_csv('feature_full_v1_457_new.csv', usecols=list(origin + predictors[(k-1)*100:k*100] + ['instance_id', 'is_trade', 'day']))
    train = train[train.day != 7]
    
    for i in enumerate(predictors[(k-1)*100:k*100]):
        if i[1] not in feature_peoformance.keys():
            tmp_predictors = origin.copy()
            tmp_predictors.append(i[1])
            xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

            cv = xgb.cv(
                params,
                dtrain=xgbtrain,
                num_boost_round=400,
                nfold=3,
                early_stopping_rounds=50,
                seed=201803)
    
            feature_performance[i[1]] = cv.iloc[-1, :]['test-logloss-mean']
            print(str(i[0]) + ':' + str(i[1]) + '->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))
            
    del train
    gc.collect()

0:hour_user_gender_id_bayes_rate_2->400:0.0534316666667
1:item_city_id_shop_review_num_level_bayes_rate_2->400:0.053474
2:item_pv_stair_bayes_rate_1->400:0.053606
3:context_page_id_isnew->400:0.0536096666667
4:user_query_queryitem_c_similarity_rank2->400:0.0531643333333
5:context_page_stair_item_price_stair_bayes_rate_1->400:0.0536136666667
6:item_id_shop_review_num_level_bayes_rate_1->400:0.052876
7:category_1->400:0.0535046666667
8:item_price_stair_shop_star_level_bayes_rate_2->400:0.0536163333333
9:context_page_stair_bayes_rate_1->400:0.0536066666667
10:category_col_58->400:0.0536126666667
11:query_categroy_count->400:0.0536186666667
12:property_1->400:0.0535853333333
13:item_avg_age->400:0.0535983333333
14:user_query_max_score10->400:0.0535453333333
15:user_query_max_score5_rank->400:0.053408
16:shop_id_user_age_stair_bayes_rate_1->400:0.0530313333333
17:item_sales_stair_shop_star_level_bayes_rate_1->400:0.053637
18:time_slice_item_city_id_bayes_rate_1->400:0.0534733333333
19:conte

461

0:context_page_stair_item_id_bayes_rate_2->400:0.0530436666667
1:item_collected_level_user_occupation_stair_bayes_rate_1->400:0.05362
2:context_page_id_item_price_level_bayes_rate_2->400:0.0536153333333
3:time_slice_user_gender_id_bayes_rate_1->400:0.0534223333333
4:item_city_id_user_occupation_stair_bayes_rate_1->400:0.053549


In [32]:
feature_performance

{'category_1': 0.053504666666666673,
 'category_col_33': 0.053620000000000001,
 'category_col_34': 0.053616666666666667,
 'category_col_43': 0.053613000000000001,
 'category_col_45': 0.053604000000000006,
 'category_col_51': 0.053611333333333337,
 'category_col_52': 0.05361266666666667,
 'category_col_54': 0.053615333333333327,
 'category_col_58': 0.05361266666666667,
 'category_col_60': 0.053611666666666669,
 'category_col_71': 0.053614666666666665,
 'category_col_72': 0.053606999999999995,
 'category_col_76': 0.053607999999999996,
 'category_col_84': 0.053603333333333336,
 'common_category_prob': 0.053596333333333329,
 'context_page_id_gender_filled_bayes_rate_1': 0.053604666666666669,
 'context_page_id_isnew': 0.053609666666666667,
 'context_page_id_item_city_id_bayes_rate_1': 0.053555999999999999,
 'context_page_id_item_price_level_bayes_rate_1': 0.053610333333333336,
 'context_page_id_item_price_level_bayes_rate_2': 0.053615333333333327,
 'context_page_id_item_price_stair_bayes_ra

In [26]:
feature_performance

{'brand_avg_age': 0.053586666666666671,
 'brand_avg_star': 0.053583666666666668,
 'brand_current_hour_percent': 0.053541999999999999,
 'brand_current_hour_query': 0.053544666666666664,
 'brand_today_query': 0.053530666666666664,
 'category_2': 0.053563,
 'category_col_0': 0.053609666666666667,
 'category_col_1': 0.053609333333333335,
 'category_col_10': 0.053606333333333332,
 'category_col_11': 0.053609999999999998,
 'category_col_12': 0.05361966666666667,
 'category_col_13': 0.053613333333333325,
 'category_col_14': 0.053613666666666671,
 'category_col_15': 0.053610666666666668,
 'category_col_16': 0.053616333333333328,
 'category_col_17': 0.053609666666666667,
 'category_col_18': 0.053607666666666665,
 'category_col_19': 0.053593333333333333,
 'category_col_2': 0.053609666666666667,
 'category_col_20': 0.05361433333333334,
 'category_col_21': 0.053615333333333327,
 'category_col_22': 0.05361966666666667,
 'category_col_23': 0.053609666666666667,
 'category_col_24': 0.0536096666666666

In [1]:
feature_performance = {'brand_avg_age': 0.053586666666666671,
 'brand_avg_star': 0.053583666666666668,
 'brand_current_hour_percent': 0.053541999999999999,
 'brand_current_hour_query': 0.053544666666666664,
 'brand_today_query': 0.053530666666666664,
 'category_2': 0.053563,
 'category_col_0': 0.053609666666666667,
 'category_col_1': 0.053609333333333335,
 'category_col_10': 0.053606333333333332,
 'category_col_11': 0.053609999999999998,
 'category_col_12': 0.05361966666666667,
 'category_col_13': 0.053613333333333325,
 'category_col_14': 0.053613666666666671,
 'category_col_15': 0.053610666666666668,
 'category_col_16': 0.053616333333333328,
 'category_col_17': 0.053609666666666667,
 'category_col_18': 0.053607666666666665,
 'category_col_19': 0.053593333333333333,
 'category_col_2': 0.053609666666666667,
 'category_col_20': 0.05361433333333334,
 'category_col_21': 0.053615333333333327,
 'category_col_22': 0.05361966666666667,
 'category_col_23': 0.053609666666666667,
 'category_col_24': 0.053609666666666667,
 'category_col_25': 0.053618666666666669,
 'category_col_26': 0.053615666666666673,
 'category_col_27': 0.053609666666666667,
 'category_col_28': 0.053615666666666673,
 'category_col_29': 0.053622333333333334,
 'category_col_3': 0.053609666666666667,
 'category_col_30': 0.053609666666666667,
 'category_col_31': 0.053599000000000001,
 'category_col_32': 0.053615999999999997,
 'category_col_35': 0.053609666666666667,
 'category_col_36': 0.053606999999999995,
 'category_col_37': 0.053615999999999997,
 'category_col_38': 0.053610999999999999,
 'category_col_39': 0.053617999999999999,
 'category_col_4': 0.053615999999999997,
 'category_col_40': 0.053608666666666666,
 'category_col_41': 0.053606999999999995,
 'category_col_42': 0.05361433333333334,
 'category_col_44': 0.053613333333333339,
 'category_col_46': 0.053611333333333337,
 'category_col_47': 0.053585333333333339,
 'category_col_48': 0.053614666666666665,
 'category_col_49': 0.053611666666666669,
 'category_col_5': 0.053609666666666667,
 'category_col_50': 0.053614999999999996,
 'category_col_53': 0.053614333333333326,
 'category_col_55': 0.053609666666666667,
 'category_col_56': 0.053603333333333336,
 'category_col_57': 0.053609666666666667,
 'category_col_59': 0.053605333333333338,
 'category_col_6': 0.053619333333333331,
 'category_col_61': 0.053609666666666667,
 'category_col_62': 0.053616666666666667,
 'category_col_63': 0.053613999999999995,
 'category_col_64': 0.053613000000000001,
 'category_col_65': 0.053609666666666667,
 'category_col_66': 0.053609666666666667,
 'category_col_67': 0.05361433333333334,
 'category_col_68': 0.05361433333333334,
 'category_col_69': 0.053615333333333327,
 'category_col_7': 0.053610666666666668,
 'category_col_70': 0.053615333333333327,
 'category_col_73': 0.053609999999999998,
 'category_col_74': 0.053609666666666667,
 'category_col_75': 0.053615999999999997,
 'category_col_77': 0.053609666666666667,
 'category_col_78': 0.053595999999999998,
 'category_col_79': 0.053613000000000001,
 'category_col_8': 0.053613666666666671,
 'category_col_80': 0.053617333333333329,
 'category_col_81': 0.053609666666666667,
 'category_col_83': 0.05360566666666667,
 'category_col_85': 0.053609666666666667,
 'category_col_86': 0.053613000000000001,
 'category_col_87': 0.053613000000000001,
 'category_col_88': 0.053610666666666668,
 'category_col_89': 0.053614333333333326,
 'category_col_9': 0.053609999999999998,
 'category_col_90': 0.053611666666666669,
 'category_col_91': 0.053617000000000005,
 'category_count': 0.053608999999999997,
 'common_category': 0.053595333333333335,
 'common_category_prob2': 0.053584333333333324,
 'common_property': 0.053613000000000001,
 'common_property_prob': 0.053621000000000002,
 'common_property_prob2': 0.053617333333333329,
 'context_id': 0.05363366666666667,
 'context_page_id_bayes_rate_1': 0.053611333333333323,
 'context_page_id_bayes_rate_2': 0.053612333333333338,
 'context_page_id_gender_filled_bayes_rate_2': 0.053609333333333335,
 'context_page_id_item_brand_id_bayes_rate_1': 0.053395666666666675,
 'context_page_id_item_brand_id_bayes_rate_2': 0.053397,
 'context_page_id_item_city_id_bayes_rate_2': 0.05355766666666667,
 'context_page_id_item_collected_level_bayes_rate_1': 0.053614999999999996,
 'context_page_id_item_collected_level_bayes_rate_2': 0.053623333333333328,
 'context_page_id_item_id_bayes_rate_1': 0.053269666666666667,
 'context_page_id_item_id_bayes_rate_2': 0.053263666666666674,
 'context_page_id_item_price_stair_bayes_rate_1': 0.053610666666666668,
 'context_page_id_item_pv_level_bayes_rate_1': 0.053620666666666671,
 'context_page_id_item_pv_level_bayes_rate_2': 0.053621333333333333,
 'context_page_id_item_pv_stair_bayes_rate_1': 0.053609333333333335,
 'context_page_id_item_pv_stair_bayes_rate_2': 0.053612666666666663,
 'context_page_id_item_sales_level_bayes_rate_1': 0.053620333333333332,
 'context_page_id_item_sales_level_bayes_rate_2': 0.053618666666666669,
 'context_page_id_item_sales_stair_bayes_rate_1': 0.053609333333333335,
 'context_page_id_item_sales_stair_bayes_rate_2': 0.053614666666666665,
 'context_page_id_shop_id_bayes_rate_1': 0.053258,
 'context_page_id_shop_review_num_level_bayes_rate_1': 0.053624333333333329,
 'context_page_id_shop_review_num_level_bayes_rate_2': 0.053628333333333333,
 'context_page_id_shop_star_level_bayes_rate_2': 0.053622000000000003,
 'context_page_id_user_age_level_bayes_rate_1': 0.053617999999999999,
 'context_page_id_user_age_level_bayes_rate_2': 0.053615333333333327,
 'context_page_id_user_age_stair_bayes_rate_1': 0.05361266666666667,
 'context_page_id_user_age_stair_bayes_rate_2': 0.053602333333333335,
 'context_page_id_user_gender_id_bayes_rate_2': 0.053607333333333333,
 'context_page_id_user_id_bayes_rate_1': 0.053515,
 'context_page_id_user_id_bayes_rate_2': 0.053512000000000004,
 'context_page_id_user_occupation_id_bayes_rate_1': 0.053621666666666672,
 'context_page_id_user_occupation_stair_bayes_rate_2': 0.053622666666666673,
 'context_page_id_user_star_level_bayes_rate_1': 0.053622333333333327,
 'context_page_id_user_star_stair_bayes_rate_1': 0.053612333333333338,
 'context_page_id_user_star_stair_bayes_rate_2': 0.053604999999999993,
 'context_page_stair': 0.05361966666666667,
 'context_page_stair_bayes_rate_2': 0.053606666666666664,
 'context_page_stair_gender_filled_bayes_rate_1': 0.053608333333333334,
 'context_page_stair_gender_filled_bayes_rate_2': 0.053608333333333334,
 'context_page_stair_item_brand_id_bayes_rate_2': 0.053259000000000001,
 'context_page_stair_item_city_id_bayes_rate_1': 0.053541000000000005,
 'context_page_stair_item_city_id_bayes_rate_2': 0.053547000000000004,
 'context_page_stair_item_collected_level_bayes_rate_1': 0.053615333333333327,
 'context_page_stair_item_collected_level_bayes_rate_2': 0.053613333333333339,
 'context_page_stair_item_id_bayes_rate_1': 0.053043333333333331,
 'context_page_stair_item_price_level_bayes_rate_1': 0.053606000000000008,
 'context_page_stair_item_price_level_bayes_rate_2': 0.053606999999999995,
 'context_page_stair_item_price_stair_bayes_rate_2': 0.053613666666666664,
 'context_page_stair_item_pv_level_bayes_rate_1': 0.053619333333333331,
 'context_page_stair_item_pv_level_bayes_rate_2': 0.053616999999999998,
 'context_page_stair_item_pv_stair_bayes_rate_1': 0.053606333333333332,
 'context_page_stair_item_sales_level_bayes_rate_1': 0.053608999999999997,
 'context_page_stair_item_sales_level_bayes_rate_2': 0.053608999999999997,
 'context_page_stair_item_sales_stair_bayes_rate_1': 0.053612666666666663,
 'context_page_stair_shop_id_bayes_rate_1': 0.053028666666666668,
 'context_page_stair_shop_id_bayes_rate_2': 0.053027666666666667,
 'context_page_stair_shop_review_num_level_bayes_rate_1': 0.053614666666666672,
 'context_page_stair_shop_review_num_level_bayes_rate_2': 0.053620666666666671,
 'context_page_stair_shop_star_level_bayes_rate_1': 0.053627666666666664,
 'context_page_stair_shop_star_level_bayes_rate_2': 0.053627333333333332,
 'context_page_stair_user_age_level_bayes_rate_1': 0.053616999999999998,
 'context_page_stair_user_age_level_bayes_rate_2': 0.053617666666666668,
 'context_page_stair_user_age_stair_bayes_rate_1': 0.053609333333333335,
 'context_page_stair_user_age_stair_bayes_rate_2': 0.053609333333333335,
 'context_page_stair_user_gender_id_bayes_rate_1': 0.05361433333333334,
 'context_page_stair_user_gender_id_bayes_rate_2': 0.053610666666666661,
 'context_page_stair_user_id_bayes_rate_1': 0.053456666666666673,
 'context_page_stair_user_id_bayes_rate_2': 0.053452,
 'context_page_stair_user_occupation_id_bayes_rate_2': 0.053620333333333332,
 'context_page_stair_user_star_level_bayes_rate_1': 0.053624333333333336,
 'context_page_stair_user_star_stair_bayes_rate_1': 0.053616666666666667,
 'context_page_stair_user_star_stair_bayes_rate_2': 0.053616666666666667,
 'context_timestamp': 0.053445666666666669,
 'gender_filled': 0.053608333333333334,
 'gender_filled_bayes_rate_1': 0.053610333333333336,
 'gender_filled_bayes_rate_2': 0.053610333333333336,
 'gender_occ': 0.053613666666666671,
 'gender_star': 0.053616000000000004,
 'hour': 0.053432666666666663,
 'hour_gender_filled_bayes_rate_1': 0.053428000000000003,
 'hour_item_brand_id_bayes_rate_1': 0.053445666666666669,
 'hour_item_brand_id_bayes_rate_2': 0.053440333333333333,
 'hour_item_city_id_bayes_rate_1': 0.053447000000000001,
 'hour_item_city_id_bayes_rate_2': 0.053445999999999994,
 'hour_item_id_bayes_rate_1': 0.053447666666666664,
 'hour_item_price_stair_bayes_rate_1': 0.053434999999999989,
 'hour_item_pv_level_bayes_rate_1': 0.053447333333333326,
 'hour_item_pv_stair_bayes_rate_2': 0.053435333333333335,
 'hour_item_sales_level_bayes_rate_1': 0.053453000000000001,
 'hour_item_sales_stair_bayes_rate_1': 0.053446,
 'hour_shop_review_num_level_bayes_rate_1': 0.053475666666666664,
 'hour_shop_review_num_level_bayes_rate_2': 0.053468999999999996,
 'hour_shop_star_level_bayes_rate_1': 0.053471333333333336,
 'hour_shop_star_level_bayes_rate_2': 0.053473,
 'hour_user_age_level_bayes_rate_1': 0.053438666666666662,
 'hour_user_age_level_bayes_rate_2': 0.053442666666666666,
 'hour_user_age_stair_bayes_rate_1': 0.053435333333333335,
 'hour_user_age_stair_bayes_rate_2': 0.053441666666666665,
 'hour_user_gender_id_bayes_rate_1': 0.053427333333333327,
 'hour_user_id_bayes_rate_1': 0.053579999999999996,
 'hour_user_occupation_id_bayes_rate_1': 0.053446666666666663,
 'hour_user_occupation_id_bayes_rate_2': 0.053442999999999997,
 'hour_user_occupation_stair_bayes_rate_1': 0.053446666666666663,
 'hour_user_occupation_stair_bayes_rate_2': 0.053447000000000001,
 'hour_user_star_level_bayes_rate_1': 0.053463333333333335,
 'hour_user_star_level_bayes_rate_2': 0.053463000000000004,
 'hour_user_star_stair_bayes_rate_1': 0.053444333333333337,
 'hour_user_star_stair_bayes_rate_2': 0.053441666666666665,
 'instance_id': 0.053629000000000003,
 'is_trade': 3.0000000000000001e-06,
 'item_avg_star': 0.053599000000000008,
 'item_brand_collect_category_rank': 0.053580666666666665,
 'item_brand_id': 0.053604000000000006,
 'item_brand_id_bayes_rate_1': 0.053151666666666673,
 'item_brand_id_bayes_rate_2': 0.053154666666666663,
 'item_brand_id_gender_filled_bayes_rate_1': 0.05317966666666666,
 'item_brand_id_gender_filled_bayes_rate_2': 0.053173666666666675,
 'item_brand_id_shop_id_bayes_rate_1': 0.052881333333333336,
 'item_brand_id_shop_id_bayes_rate_2': 0.052884333333333332,
 'item_brand_id_shop_review_num_level_bayes_rate_1': 0.053113999999999995,
 'item_brand_id_shop_review_num_level_bayes_rate_2': 0.053114999999999996,
 'item_brand_id_shop_star_level_bayes_rate_1': 0.05311433333333334,
 'item_brand_id_shop_star_level_bayes_rate_2': 0.053111666666666668,
 'item_brand_id_user_age_level_bayes_rate_1': 0.053376666666666663,
 'item_brand_id_user_age_level_bayes_rate_2': 0.053380000000000004,
 'item_brand_id_user_age_level_percent': 0.053616333333333342,
 'item_brand_id_user_age_stair_bayes_rate_1': 0.053255333333333328,
 'item_brand_id_user_age_stair_bayes_rate_2': 0.053255999999999998,
 'item_brand_id_user_gender_id_bayes_rate_1': 0.053256000000000005,
 'item_brand_id_user_id_bayes_rate_1': 0.053616999999999998,
 'item_brand_id_user_id_bayes_rate_2': 0.053615000000000003,
 'item_brand_id_user_occupation_id_bayes_rate_1': 0.053269666666666667,
 'item_brand_id_user_occupation_id_bayes_rate_2': 0.053277666666666668,
 'item_brand_id_user_occupation_id_percent': 0.053608333333333334,
 'item_brand_id_user_occupation_stair_bayes_rate_2': 0.053227000000000003,
 'item_brand_id_user_star_level_bayes_rate_1': 0.053439333333333339,
 'item_brand_id_user_star_level_bayes_rate_2': 0.053438999999999993,
 'item_brand_id_user_star_level_percent': 0.053614333333333326,
 'item_brand_id_user_star_stair_bayes_rate_1': 0.053187666666666668,
 'item_brand_id_user_star_stair_bayes_rate_2': 0.053196333333333339,
 'item_brand_price_category_percent': 0.053563333333333331,
 'item_brand_price_category_rank': 0.053584333333333324,
 'item_brand_pv_category_percent': 0.053596333333333329,
 'item_brand_pv_category_rank': 0.053563333333333331,
 'item_brand_sales_category_percent': 0.053581333333333335,
 'item_brand_sales_category_rank': 0.053578000000000008,
 'item_brand_sales_div_pv_category_percent': 0.053590666666666675,
 'item_brand_sales_div_pv_category_rank': 0.053583333333333337,
 'item_city_collect_category_percent': 0.053577666666666669,
 'item_city_collect_category_rank': 0.053574333333333335,
 'item_city_id': 0.053552999999999996,
 'item_city_id_bayes_rate_1': 0.053523333333333332,
 'item_city_id_gender_filled_bayes_rate_2': 0.053525333333333334,
 'item_city_id_isnew': 0.053609666666666667,
 'item_city_id_shop_id_bayes_rate_1': 0.052870333333333332,
 'item_city_id_shop_id_bayes_rate_2': 0.052867999999999998,
 'item_city_id_shop_star_level_bayes_rate_1': 0.053463666666666666,
 'item_city_id_shop_star_level_bayes_rate_2': 0.053468333333333333,
 'item_city_id_user_age_level_bayes_rate_1': 0.053550999999999994,
 'item_city_id_user_age_level_bayes_rate_2': 0.053552333333333334,
 'item_city_id_user_age_stair_bayes_rate_1': 0.053537000000000001,
 'item_city_id_user_age_stair_bayes_rate_2': 0.053539666666666673,
 'item_city_id_user_gender_id_bayes_rate_1': 0.053533999999999998,
 'item_city_id_user_gender_id_bayes_rate_2': 0.053537000000000001,
 'item_city_id_user_id_bayes_rate_1': 0.053606333333333332,
 'item_city_id_user_id_bayes_rate_2': 0.053603333333333336,
 'item_city_id_user_occupation_id_bayes_rate_1': 0.05354133333333333,
 'item_city_id_user_occupation_id_bayes_rate_2': 0.053543000000000007,
 'item_city_id_user_occupation_stair_bayes_rate_2': 0.053545000000000002,
 'item_city_id_user_star_level_bayes_rate_1': 0.053555999999999993,
 'item_city_id_user_star_level_bayes_rate_2': 0.053556333333333338,
 'item_city_id_user_star_stair_bayes_rate_1': 0.053528666666666669,
 'item_city_id_user_star_stair_bayes_rate_2': 0.053526999999999998,
 'item_city_pv_category_percent': 0.053566999999999997,
 'item_city_sales_category_percent': 0.053555666666666668,
 'item_city_sales_category_rank': 0.053578333333333339,
 'item_city_sales_div_pv_category_percent': 0.05357,
 'item_city_sales_div_pv_category_rank': 0.053579999999999996,
 'item_cnt_total': 0.053610333333333336,
 'item_collect_category_percent': 0.053584666666666669,
 'item_collect_category_rank': 0.053577333333333338,
 'item_collect_div_pv': 0.053606666666666664,
 'item_collected_level_bayes_rate_1': 0.053611666666666669,
 'item_collected_level_bayes_rate_2': 0.053611666666666669,
 'item_collected_level_gender_filled_bayes_rate_1': 0.053606666666666664,
 'item_collected_level_gender_filled_bayes_rate_2': 0.053610333333333336,
 'item_collected_level_shop_id_bayes_rate_1': 0.052937666666666661,
 'item_collected_level_shop_id_bayes_rate_2': 0.052942999999999997,
 'item_collected_level_shop_review_num_level_bayes_rate_1': 0.053601666666666666,
 'item_collected_level_shop_review_num_level_bayes_rate_2': 0.053602333333333335,
 'item_collected_level_shop_star_level_bayes_rate_1': 0.053605000000000007,
 'item_collected_level_shop_star_level_bayes_rate_2': 0.053605333333333338,
 'item_collected_level_user_age_level_bayes_rate_1': 0.053625666666666662,
 'item_collected_level_user_age_stair_bayes_rate_2': 0.05361433333333334,
 'item_collected_level_user_gender_id_bayes_rate_2': 0.053617666666666668,
 'item_collected_level_user_id_bayes_rate_2': 0.053568666666666674,
 'item_collected_level_user_occupation_id_bayes_rate_1': 0.053613333333333339,
 'item_collected_level_user_occupation_id_bayes_rate_2': 0.053615333333333327,
 'item_collected_level_user_occupation_stair_bayes_rate_2': 0.053619,
 'item_collected_level_user_star_level_bayes_rate_1': 0.053621666666666672,
 'item_collected_level_user_star_level_bayes_rate_2': 0.053623333333333335,
 'item_collected_level_user_star_stair_bayes_rate_1': 0.053612999999999994,
 'item_collected_level_user_star_stair_bayes_rate_2': 0.053614333333333326,
 'item_current_hour_percent': 0.053611333333333337,
 'item_current_hour_query': 0.053596666666666661,
 'item_id': 0.053606000000000008,
 'item_id_bayes_rate_1': 0.052852666666666659,
 'item_id_bayes_rate_2': 0.052853333333333335,
 'item_id_gender_filled_bayes_rate_2': 0.052891333333333339,
 'item_id_isnew': 0.053612333333333338,
 'item_id_shop_id_bayes_rate_2': 0.052853333333333335,
 'item_id_shop_review_num_level_bayes_rate_2': 0.052867999999999998,
 'item_id_shop_star_level_bayes_rate_1': 0.052866000000000003,
 'item_id_shop_star_level_bayes_rate_2': 0.05286833333333333,
 'item_id_user_age_level_bayes_rate_1': 0.053284333333333329,
 'item_id_user_age_level_bayes_rate_2': 0.053283999999999998,
 'item_id_user_age_level_percent': 0.053608999999999997,
 'item_id_user_age_stair_bayes_rate_1': 0.053065333333333332,
 'item_id_user_age_stair_bayes_rate_2': 0.053061666666666667,
 'item_id_user_gender_id_bayes_rate_1': 0.053043,
 'item_id_user_gender_id_bayes_rate_2': 0.053047333333333335,
 'item_id_user_id_bayes_rate_2': 0.05358566666666667,
 'item_id_user_occupation_id_bayes_rate_1': 0.053122666666666672,
 'item_id_user_occupation_id_bayes_rate_2': 0.053120000000000001,
 'item_id_user_occupation_id_percent': 0.053612666666666663,
 'item_id_user_occupation_stair_bayes_rate_1': 0.053047000000000004,
 'item_id_user_occupation_stair_bayes_rate_2': 0.053041666666666661,
 'item_id_user_star_level_bayes_rate_1': 0.053377000000000001,
 'item_id_user_star_level_bayes_rate_2': 0.053374000000000005,
 'item_id_user_star_level_percent': 0.053615000000000003,
 'item_id_user_star_stair_bayes_rate_1': 0.052914666666666665,
 'item_price_category_rank': 0.053546666666666666,
 'item_price_level_bayes_rate_1': 0.053598,
 'item_price_level_bayes_rate_2': 0.053598,
 'item_price_level_gender_filled_bayes_rate_1': 0.053603333333333336,
 'item_price_level_gender_filled_bayes_rate_2': 0.053603333333333336,
 'item_price_level_shop_id_bayes_rate_1': 0.052836666666666664,
 'item_price_level_shop_id_bayes_rate_2': 0.052836333333333339,
 'item_price_level_shop_review_num_level_bayes_rate_1': 0.053607333333333333,
 'item_price_level_shop_review_num_level_bayes_rate_2': 0.053608999999999997,
 'item_price_level_shop_star_level_bayes_rate_1': 0.053610999999999999,
 'item_price_level_user_age_level_bayes_rate_1': 0.053602000000000004,
 'item_price_level_user_age_level_bayes_rate_2': 0.053611999999999993,
 'item_price_level_user_age_stair_bayes_rate_1': 0.053595000000000004,
 'item_price_level_user_age_stair_bayes_rate_2': 0.053595000000000004,
 'item_price_level_user_gender_id_bayes_rate_1': 0.053599000000000001,
 'item_price_level_user_gender_id_bayes_rate_2': 0.053599000000000001,
 'item_price_level_user_occupation_id_bayes_rate_2': 0.053609999999999991,
 'item_price_level_user_occupation_stair_bayes_rate_1': 0.053615999999999997,
 'item_price_level_user_occupation_stair_bayes_rate_2': 0.053615999999999997,
 'item_price_level_user_star_level_bayes_rate_1': 0.053622666666666673,
 'item_price_level_user_star_stair_bayes_rate_1': 0.053600666666666665,
 'item_price_level_user_star_stair_bayes_rate_2': 0.053600666666666665,
 'item_price_stair': 0.053616333333333328,
 'item_price_stair_bayes_rate_1': 0.053605333333333338,
 'item_price_stair_bayes_rate_2': 0.053605333333333338,
 'item_price_stair_gender_filled_bayes_rate_2': 0.053602666666666667,
 'item_price_stair_shop_id_bayes_rate_1': 0.052850666666666664,
 'item_price_stair_shop_id_bayes_rate_2': 0.052852666666666659,
 'item_price_stair_shop_review_num_level_bayes_rate_1': 0.053606666666666671,
 'item_price_stair_shop_review_num_level_bayes_rate_2': 0.053606333333333332,
 'item_price_stair_user_age_level_bayes_rate_2': 0.053600000000000002,
 'item_price_stair_user_age_stair_bayes_rate_1': 0.053601333333333334,
 'item_price_stair_user_age_stair_bayes_rate_2': 0.053601333333333334,
 'item_price_stair_user_gender_id_bayes_rate_2': 0.053608666666666666,
 'item_price_stair_user_id_bayes_rate_1': 0.053463666666666666,
 'item_price_stair_user_id_bayes_rate_2': 0.053470999999999998,
 'item_price_stair_user_occupation_id_bayes_rate_1': 0.053602333333333335,
 'item_price_stair_user_occupation_id_bayes_rate_2': 0.053602333333333335,
 'item_price_stair_user_occupation_stair_bayes_rate_1': 0.053608333333333334,
 'item_price_stair_user_occupation_stair_bayes_rate_2': 0.053608333333333334,
 'item_price_stair_user_star_level_bayes_rate_2': 0.053626333333333331,
 'item_price_stair_user_star_stair_bayes_rate_1': 0.053604999999999993,
 'item_price_stair_user_star_stair_bayes_rate_2': 0.053604999999999993,
 'item_pv_category_percent': 0.053593000000000002,
 'item_pv_category_rank': 0.053592000000000001,
 'item_pv_level_bayes_rate_1': 0.053614000000000002,
 'item_pv_level_bayes_rate_2': 0.053606999999999995,
 'item_pv_level_gender_filled_bayes_rate_1': 0.053614999999999996,
 'item_pv_level_gender_filled_bayes_rate_2': 0.053613666666666664,
 'item_pv_level_shop_review_num_level_bayes_rate_2': 0.053620333333333332,
 'item_pv_level_shop_star_level_bayes_rate_1': 0.053622333333333327,
 'item_pv_level_user_age_level_bayes_rate_1': 0.053628000000000002,
 'item_pv_level_user_age_level_bayes_rate_2': 0.053623333333333328,
 'item_pv_level_user_age_stair_bayes_rate_1': 0.053622000000000003,
 'item_pv_level_user_gender_id_bayes_rate_1': 0.053619333333333331,
 'item_pv_level_user_gender_id_bayes_rate_2': 0.053618666666666669,
 'item_pv_level_user_id_bayes_rate_1': 0.053571333333333332,
 'item_pv_level_user_occupation_id_bayes_rate_1': 0.053627000000000001,
 'item_pv_level_user_occupation_id_bayes_rate_2': 0.053622333333333327,
 'item_pv_level_user_occupation_stair_bayes_rate_1': 0.053622999999999997,
 'item_pv_level_user_occupation_stair_bayes_rate_2': 0.053621333333333333,
 'item_pv_level_user_star_level_bayes_rate_1': 0.053631000000000005,
 'item_pv_level_user_star_stair_bayes_rate_1': 0.053617666666666668,
 'item_pv_level_user_star_stair_bayes_rate_2': 0.053616999999999998,
 'item_pv_stair': 0.053619333333333331,
 'item_pv_stair_gender_filled_bayes_rate_2': 0.053603666666666661,
 'item_pv_stair_shop_id_bayes_rate_1': 0.052883333333333338,
 'item_pv_stair_shop_id_bayes_rate_2': 0.052880000000000003,
 'item_pv_stair_shop_review_num_level_bayes_rate_2': 0.053608666666666666,
 'item_pv_stair_shop_star_level_bayes_rate_1': 0.053606333333333332,
 'item_pv_stair_shop_star_level_bayes_rate_2': 0.053606333333333332,
 'item_pv_stair_user_age_level_bayes_rate_1': 0.053606333333333332,
 'item_pv_stair_user_age_level_bayes_rate_2': 0.053606000000000008,
 'item_pv_stair_user_age_stair_bayes_rate_1': 0.053606000000000008,
 'item_pv_stair_user_age_stair_bayes_rate_2': 0.053606000000000008,
 'item_pv_stair_user_gender_id_bayes_rate_1': 0.053610999999999999,
 'item_pv_stair_user_gender_id_bayes_rate_2': 0.053610999999999999,
 'item_pv_stair_user_id_bayes_rate_1': 0.05340799999999999,
 'item_pv_stair_user_id_bayes_rate_2': 0.053407000000000003,
 'item_pv_stair_user_occupation_stair_bayes_rate_2': 0.053621333333333333,
 'item_pv_stair_user_star_level_bayes_rate_2': 0.053615000000000003,
 'item_pv_stair_user_star_stair_bayes_rate_1': 0.053604666666666669,
 'item_pv_stair_user_star_stair_bayes_rate_2': 0.053604666666666669,
 'item_sales_category_percent': 0.053586666666666664,
 'item_sales_category_rank': 0.053595000000000004,
 'item_sales_div_pv_rank': 0.053590666666666668,
 'item_sales_level_bayes_rate_1': 0.053609333333333335,
 'item_sales_level_bayes_rate_2': 0.053609333333333335,
 'item_sales_level_gender_filled_bayes_rate_1': 0.053609999999999998,
 'item_sales_level_gender_filled_bayes_rate_2': 0.053609999999999998,
 'item_sales_level_shop_id_bayes_rate_1': 0.052946,
 'item_sales_level_shop_id_bayes_rate_2': 0.052947666666666671,
 'item_sales_level_shop_review_num_level_bayes_rate_1': 0.053666333333333337,
 'item_sales_level_shop_star_level_bayes_rate_1': 0.053674333333333331,
 'item_sales_level_shop_star_level_bayes_rate_2': 0.053668333333333339,
 'item_sales_level_user_age_level_bayes_rate_1': 0.053623333333333328,
 'item_sales_level_user_age_level_bayes_rate_2': 0.053621333333333333,
 'item_sales_level_user_age_stair_bayes_rate_1': 0.053612666666666663,
 'item_sales_level_user_age_stair_bayes_rate_2': 0.053612666666666663,
 'item_sales_level_user_gender_id_bayes_rate_1': 0.053605333333333338,
 'item_sales_level_user_gender_id_bayes_rate_2': 0.053605000000000007,
 'item_sales_level_user_id_bayes_rate_1': 0.053583333333333337,
 'item_sales_level_user_occupation_id_bayes_rate_1': 0.05361833333333333,
 'item_sales_level_user_occupation_id_bayes_rate_2': 0.053624999999999999,
 'item_sales_level_user_occupation_stair_bayes_rate_1': 0.053606999999999995,
 'item_sales_level_user_star_level_bayes_rate_1': 0.053640333333333338,
 'item_sales_level_user_star_level_bayes_rate_2': 0.053638333333333336,
 'item_sales_level_user_star_stair_bayes_rate_1': 0.053613999999999995,
 'item_sales_level_user_star_stair_bayes_rate_2': 0.053613999999999995,
 'item_sales_stair': 0.053617999999999999,
 'item_sales_stair_bayes_rate_1': 0.053616333333333328,
 'item_sales_stair_gender_filled_bayes_rate_1': 0.053611333333333337,
 'item_sales_stair_gender_filled_bayes_rate_2': 0.053611333333333337,
 'item_sales_stair_shop_id_bayes_rate_1': 0.052872333333333334,
 'item_sales_stair_shop_id_bayes_rate_2': 0.052868666666666668,
 'item_sales_stair_shop_review_num_level_bayes_rate_1': 0.053635999999999996,
 'item_sales_stair_shop_review_num_level_bayes_rate_2': 0.053637000000000011,
 'item_sales_stair_shop_star_level_bayes_rate_2': 0.053637000000000011,
 'item_sales_stair_user_age_level_bayes_rate_1': 0.053617666666666668,
 'item_sales_stair_user_age_level_bayes_rate_2': 0.053617666666666668,
 'item_sales_stair_user_age_stair_bayes_rate_1': 0.05361966666666667,
 'item_sales_stair_user_age_stair_bayes_rate_2': 0.05361966666666667,
 'item_sales_stair_user_gender_id_bayes_rate_1': 0.053611999999999993,
 'item_sales_stair_user_gender_id_bayes_rate_2': 0.053611999999999993,
 'item_sales_stair_user_id_bayes_rate_1': 0.053513999999999999,
 'item_sales_stair_user_id_bayes_rate_2': 0.053512333333333335,
 'item_sales_stair_user_occupation_id_bayes_rate_1': 0.053622000000000003,
 'item_sales_stair_user_occupation_id_bayes_rate_2': 0.053622000000000003,
 'item_sales_stair_user_occupation_stair_bayes_rate_1': 0.053624999999999999,
 'item_sales_stair_user_occupation_stair_bayes_rate_2': 0.053624999999999999,
 'item_sales_stair_user_star_level_bayes_rate_1': 0.05362533333333333,
 'item_sales_stair_user_star_level_bayes_rate_2': 0.053622333333333327,
 'item_sales_stair_user_star_stair_bayes_rate_1': 0.053610333333333336,
 'item_sales_stair_user_star_stair_bayes_rate_2': 0.053610333333333336,
 'item_today_query_percent': 0.053606666666666664,
 'mins': 0.053625666666666662,
 'price_collect': 0.053616666666666667,
 'property_0': 0.053566666666666672,
 'property_10': 0.053602000000000004,
 'property_11': 0.053599000000000001,
 'property_12': 0.05359733333333333,
 'property_13': 0.053610666666666661,
 'property_14': 0.053602666666666667,
 'property_15': 0.053612,
 'property_16': 0.053608333333333334,
 'property_17': 0.053607333333333333,
 'property_18': 0.053610666666666668,
 'property_19': 0.053600666666666665,
 'property_2': 0.053591,
 'property_20': 0.053606000000000008,
 'property_22': 0.053596999999999999,
 'property_25': 0.053585666666666663,
 'property_27': 0.053573000000000003,
 'property_28': 0.053577666666666669,
 'property_29': 0.053577333333333331,
 'property_3': 0.053588666666666666,
 'property_30': 0.053576666666666661,
 'property_32': 0.053585000000000001,
 'property_34': 0.053588333333333328,
 'property_35': 0.053592333333333332,
 'property_36': 0.053586666666666671,
 'property_37': 0.053598666666666663,
 'property_38': 0.053588333333333328,
 'property_39': 0.053594999999999997,
 'property_4': 0.053599000000000001,
 'property_5': 0.053606666666666671,
 'property_6': 0.053597000000000006,
 'property_7': 0.053603000000000005,
 'property_8': 0.053605333333333338,
 'query_category_0': 0.053516666666666664,
 'query_category_1': 0.053588333333333328,
 'query_category_2': 0.053613000000000001,
 'query_category_3': 0.053622666666666673,
 'query_category_4': 0.053620000000000001,
 'query_category_5': 0.053617999999999999,
 'query_category_6': 0.053615999999999997,
 'query_category_7': 0.053613000000000001,
 'query_category_9': 0.053615999999999997,
 'query_col_1': 0.053613333333333339,
 'query_col_12': 0.053609666666666667,
 'query_col_13': 0.053609666666666667,
 'query_col_14': 0.053609666666666667,
 'query_col_15': 0.053609666666666667,
 'query_col_16': 0.053609666666666667,
 'query_col_17': 0.053609666666666667,
 'query_col_2': 0.053615999999999997,
 'query_col_3': 0.053617333333333329,
 'query_col_4': 0.053611333333333337,
 'query_col_5': 0.05361266666666667,
 'query_col_6': 0.053611333333333337,
 'query_col_7': 0.053608666666666666,
 'query_col_8': 0.053609666666666667,
 'query_col_9': 0.053609666666666667,
 'review_star': 0.053612666666666663,
 'sale_collect': 0.053606999999999995,
 'shop_avg_age': 0.053580999999999997,
 'shop_cnt_day': 0.053509666666666671,
 'shop_cnt_dayavg_total': 0.053493333333333337,
 'shop_cnt_total': 0.053505000000000004,
 'shop_current_hour_percent': 0.053516333333333332,
 'shop_id': 0.053587333333333341,
 'shop_id_bayes_rate_2': 0.052872333333333334,
 'shop_id_gender_filled_bayes_rate_1': 0.052901333333333335,
 'shop_id_gender_filled_bayes_rate_2': 0.052901333333333335,
 'shop_id_user_age_level_bayes_rate_2': 0.053230666666666669,
 'shop_id_user_age_level_percent': 0.053596666666666674,
 'shop_id_user_age_stair_bayes_rate_2': 0.053025999999999997,
 'shop_id_user_gender_id_bayes_rate_1': 0.05305066666666667,
 'shop_id_user_gender_id_bayes_rate_2': 0.053045000000000002,
 'shop_id_user_gender_id_percent': 0.053575333333333336,
 'shop_id_user_id_bayes_rate_1': 0.053588999999999998,
 'shop_id_user_id_bayes_rate_2': 0.053594666666666659,
 'shop_id_user_occupation_id_bayes_rate_1': 0.053074333333333334,
 'shop_id_user_occupation_id_bayes_rate_2': 0.053072333333333332,
 'shop_id_user_occupation_id_percent': 0.053598333333333331,
 'shop_id_user_occupation_stair_bayes_rate_2': 0.052997666666666665,
 'shop_id_user_star_level_bayes_rate_1': 0.053325333333333336,
 'shop_id_user_star_level_bayes_rate_2': 0.053325666666666667,
 'shop_id_user_star_level_percent': 0.053606999999999995,
 'shop_id_user_star_stair_bayes_rate_1': 0.052930999999999999,
 'shop_id_user_star_stair_bayes_rate_2': 0.052930999999999999,
 'shop_review_num_level_bayes_rate_1': 0.053604666666666669,
 'shop_review_num_level_user_age_level_bayes_rate_1': 0.053621000000000002,
 'shop_review_num_level_user_age_level_bayes_rate_2': 0.053618666666666669,
 'shop_review_num_level_user_age_stair_bayes_rate_1': 0.053619,
 'shop_review_num_level_user_age_stair_bayes_rate_2': 0.053618666666666669,
 'shop_review_num_level_user_gender_id_bayes_rate_1': 0.053608999999999997,
 'shop_review_num_level_user_gender_id_bayes_rate_2': 0.053613000000000001,
 'shop_review_num_level_user_id_bayes_rate_2': 0.053578333333333339,
 'shop_review_num_level_user_occupation_id_bayes_rate_1': 0.053616333333333342,
 'shop_review_num_level_user_occupation_id_bayes_rate_2': 0.053617333333333329,
 'shop_review_num_level_user_occupation_stair_bayes_rate_1': 0.053615333333333327,
 'shop_review_num_level_user_occupation_stair_bayes_rate_2': 0.053612333333333338,
 'shop_review_num_level_user_star_level_bayes_rate_1': 0.053633666666666663,
 'shop_review_num_level_user_star_level_bayes_rate_2': 0.053631333333333336,
 'shop_review_num_level_user_star_stair_bayes_rate_1': 0.053613333333333325,
 'shop_star_level_bayes_rate_1': 0.053610999999999999,
 'shop_star_level_bayes_rate_2': 0.053610999999999999,
 'shop_star_level_gender_filled_bayes_rate_2': 0.053607666666666665,
 'shop_star_level_user_age_level_bayes_rate_1': 0.053620666666666671,
 'shop_star_level_user_age_stair_bayes_rate_1': 0.053623333333333335,
 'shop_star_level_user_age_stair_bayes_rate_2': 0.053624333333333329,
 'shop_star_level_user_gender_id_bayes_rate_1': 0.053619,
 'shop_star_level_user_gender_id_bayes_rate_2': 0.053617999999999999,
 'shop_star_level_user_id_bayes_rate_1': 0.053567666666666673,
 'shop_star_level_user_occupation_id_bayes_rate_1': 0.053615666666666673,
 'shop_star_level_user_occupation_id_bayes_rate_2': 0.053616666666666667,
 'shop_star_level_user_occupation_stair_bayes_rate_1': 0.053611666666666669,
 'shop_star_level_user_occupation_stair_bayes_rate_2': 0.053615333333333327,
 'shop_star_level_user_star_level_bayes_rate_1': 0.053622666666666673,
 'shop_star_level_user_star_stair_bayes_rate_2': 0.053612666666666663,
 'shop_today_query': 0.053460666666666663,
 'time_slice': 0.053423666666666668,
 'time_slice_bayes_rate_1': 0.053429999999999998,
 'time_slice_bayes_rate_2': 0.053429999999999998,
 'time_slice_gender_filled_bayes_rate_1': 0.053420999999999996,
 'time_slice_gender_filled_bayes_rate_2': 0.053424333333333331,
 'time_slice_item_brand_id_bayes_rate_1': 0.053495666666666671,
 'time_slice_item_brand_id_bayes_rate_2': 0.053494333333333331,
 'time_slice_item_city_id_bayes_rate_2': 0.053475666666666664,
 'time_slice_item_collected_level_bayes_rate_1': 0.053466333333333338,
 'time_slice_item_collected_level_bayes_rate_2': 0.053465666666666668,
 'time_slice_item_id_bayes_rate_1': 0.053523999999999995,
 'time_slice_item_id_bayes_rate_2': 0.053533333333333329,
 'time_slice_item_price_level_bayes_rate_1': 0.053449999999999998,
 'time_slice_item_price_level_bayes_rate_2': 0.053449999999999998,
 'time_slice_item_price_stair_bayes_rate_1': 0.053422666666666667,
 'time_slice_item_price_stair_bayes_rate_2': 0.053421666666666666,
 'time_slice_item_pv_level_bayes_rate_1': 0.053468333333333333,
 'time_slice_item_pv_stair_bayes_rate_1': 0.053429999999999998,
 'time_slice_item_pv_stair_bayes_rate_2': 0.05343033333333333,
 'time_slice_item_sales_level_bayes_rate_1': 0.053452666666666669,
 'time_slice_item_sales_level_bayes_rate_2': 0.053452999999999994,
 'time_slice_item_sales_stair_bayes_rate_1': 0.053441333333333341,
 'time_slice_item_sales_stair_bayes_rate_2': 0.053444333333333337,
 'time_slice_shop_id_bayes_rate_1': 0.053479999999999993,
 'time_slice_shop_review_num_level_bayes_rate_1': 0.053488000000000001,
 'time_slice_shop_review_num_level_bayes_rate_2': 0.053482000000000002,
 'time_slice_shop_star_level_bayes_rate_1': 0.053482666666666671,
 'time_slice_shop_star_level_bayes_rate_2': 0.053481999999999995,
 'time_slice_user_age_level_bayes_rate_1': 0.053428666666666659,
 'time_slice_user_age_level_bayes_rate_2': 0.053436000000000004,
 'time_slice_user_age_stair_bayes_rate_1': 0.053432,
 'time_slice_user_id_bayes_rate_2': 0.053605333333333338,
 'time_slice_user_occupation_id_bayes_rate_1': 0.053443666666666667,
 'time_slice_user_occupation_stair_bayes_rate_1': 0.053436333333333336,
 'time_slice_user_star_level_bayes_rate_1': 0.053469666666666665,
 'time_slice_user_star_stair_bayes_rate_1': 0.053439333333333339,
 'time_slice_user_star_stair_bayes_rate_2': 0.053438000000000006,
 'user_age_level_bayes_rate_1': 0.053610333333333336,
 'user_age_level_isnew': 0.053609666666666667,
 'user_age_stair': 0.053616666666666667,
 'user_age_stair_bayes_rate_1': 0.053609333333333342,
 'user_age_stair_bayes_rate_2': 0.053609333333333342,
 'user_brand_age_dist': 0.053603333333333336,
 'user_brand_star_dist': 0.053603666666666668,
 'user_cnt_dayavg_total': 0.053504333333333341,
 'user_cnt_total': 0.053496333333333333,
 'user_current_hour_percent': 0.053382666666666669,
 'user_current_hour_query': 0.053287333333333332,
 'user_gender_id_bayes_rate_1': 0.053601666666666666,
 'user_gender_id_bayes_rate_2': 0.053601666666666666,
 'user_id': 0.053632333333333337,
 'user_id_bayes_rate_1': 0.053384999999999995,
 'user_id_bayes_rate_2': 0.05337533333333333,
 'user_item_age_dist': 0.053610333333333336,
 'user_item_star_dist': 0.053620666666666671,
 'user_lastquery': 0.052693666666666666,
 'user_lastquery_timedelta': 0.052493333333333336,
 'user_occupation_id_bayes_rate_1': 0.053612999999999994,
 'user_occupation_id_bayes_rate_2': 0.053613666666666664,
 'user_occupation_id_isnew': 0.053609666666666667,
 'user_occupation_stair': 0.053610999999999999,
 'user_occupation_stair_bayes_rate_2': 0.053609666666666667,
 'user_query_best_delivery': 0.053494333333333331,
 'user_query_best_description': 0.053490333333333334,
 'user_query_best_service': 0.053485333333333336,
 'user_query_cheapest': 0.053580333333333334,
 'user_query_collect_percent': 0.053311000000000004,
 'user_query_description_percent': 0.053393666666666666,
 'user_query_goodreview_percent': 0.053386333333333334,
 'user_query_goodreview_rank': 0.053169333333333339,
 'user_query_max_score1': 0.053377000000000008,
 'user_query_max_score10_rank': 0.053244333333333338,
 'user_query_max_score1_percent': 0.053026333333333335,
 'user_query_max_score1_rank': 0.053419666666666671,
 'user_query_max_score2': 0.053301999999999995,
 'user_query_max_score2_percent': 0.053026333333333335,
 'user_query_max_score3': 0.053387999999999998,
 'user_query_max_score3_percent': 0.052962000000000002,
 'user_query_max_score3_rank': 0.053318999999999998,
 'user_query_max_score4': 0.053507666666666655,
 'user_query_max_score4_rank': 0.053302666666666665,
 'user_query_max_score5': 0.053351666666666665,
 'user_query_max_score5_percent': 0.053024999999999996,
 'user_query_max_score6': 0.053431333333333331,
 'user_query_max_score6_percent': 0.052962000000000002,
 'user_query_max_score7': 0.053477333333333328,
 'user_query_max_score7_percent': 0.052944333333333336,
 'user_query_max_score7_rank': 0.053375000000000006,
 'user_query_max_score8': 0.053387999999999998,
 'user_query_max_score8_percent': 0.052962000000000002,
 'user_query_max_score8_rank': 0.053318999999999998,
 'user_query_max_score9': 0.053469333333333334,
 'user_query_max_score9_percent': 0.053068666666666674,
 'user_query_max_score9_rank': 0.053244333333333338,
 'user_query_maxcollect': 0.053516333333333332,
 'user_query_maxgoodreview': 0.053505333333333328,
 'user_query_maxpv': 0.053521666666666669,
 'user_query_maxqueryitem_c_similarity': 0.053558999999999996,
 'user_query_maxqueryitem_c_similarity2': 0.053565999999999996,
 'user_query_maxqueryitem_p_similarity': 0.05356466666666667,
 'user_query_maxqueryitem_p_similarity2': 0.053577000000000007,
 'user_query_maxreview': 0.053483333333333334,
 'user_query_maxsell': 0.053550333333333332,
 'user_query_price_rank': 0.053276999999999998,
 'user_query_pv_rank': 0.05323566666666666,
 'user_query_queryitem_c_similarity_percent': 0.053381666666666668,
 'user_query_queryitem_c_similarity_percent2': 0.053349333333333339,
 'user_query_queryitem_c_similarity_rank': 0.053192999999999997,
 'user_query_queryitem_p_similarity_percent': 0.053321666666666663,
 'user_query_queryitem_p_similarity_percent2': 0.053325000000000004,
 'user_query_queryitem_p_similarity_rank': 0.053199999999999997,
 'user_query_queryitem_p_similarity_rank2': 0.053198333333333341,
 'user_query_review_rank': 0.053193666666666667,
 'user_query_sale_rank': 0.053330999999999996,
 'user_query_sales_div_pv_percent': 0.053293,
 'user_query_sales_div_pv_rank': 0.053350333333333333,
 'user_query_service_percent': 0.053409000000000005,
 'user_query_service_rank': 0.053190666666666664,
 'user_query_shopstar_percent': 0.053372666666666672,
 'user_query_shopstar_rank': 0.053187333333333336,
 'user_shop_age_dist': 0.053601000000000003,
 'user_shop_dist_dist': 0.053603666666666668,
 'user_star_level_bayes_rate_1': 0.053621333333333333,
 'user_star_stair': 0.053612999999999994,
 'user_star_stair_bayes_rate_1': 0.053606999999999995,
 'user_total_query_times': 0.05308633333333334,
 'category_1': 0.053504666666666673,
 'category_col_33': 0.053620000000000001,
 'category_col_34': 0.053616666666666667,
 'category_col_43': 0.053613000000000001,
 'category_col_45': 0.053604000000000006,
 'category_col_51': 0.053611333333333337,
 'category_col_52': 0.05361266666666667,
 'category_col_54': 0.053615333333333327,
 'category_col_58': 0.05361266666666667,
 'category_col_60': 0.053611666666666669,
 'category_col_71': 0.053614666666666665,
 'category_col_72': 0.053606999999999995,
 'category_col_76': 0.053607999999999996,
 'category_col_84': 0.053603333333333336,
 'common_category_prob': 0.053596333333333329,
 'context_page_id_gender_filled_bayes_rate_1': 0.053604666666666669,
 'context_page_id_isnew': 0.053609666666666667,
 'context_page_id_item_city_id_bayes_rate_1': 0.053555999999999999,
 'context_page_id_item_price_level_bayes_rate_1': 0.053610333333333336,
 'context_page_id_item_price_level_bayes_rate_2': 0.053615333333333327,
 'context_page_id_item_price_stair_bayes_rate_2': 0.053610666666666668,
 'context_page_id_shop_id_bayes_rate_2': 0.053248666666666666,
 'context_page_id_shop_star_level_bayes_rate_1': 0.053620000000000001,
 'context_page_id_user_gender_id_bayes_rate_1': 0.053611333333333323,
 'context_page_id_user_occupation_id_bayes_rate_2': 0.053620000000000001,
 'context_page_id_user_occupation_stair_bayes_rate_1': 0.053624999999999999,
 'context_page_id_user_star_level_bayes_rate_2': 0.053622333333333327,
 'context_page_stair_bayes_rate_1': 0.053606666666666664,
 'context_page_stair_item_id_bayes_rate_2': 0.053043666666666663,
 'context_page_stair_item_price_stair_bayes_rate_1': 0.053613666666666664,
 'context_page_stair_item_pv_stair_bayes_rate_2': 0.053606333333333332,
 'context_page_stair_item_sales_stair_bayes_rate_2': 0.053612666666666663,
 'context_page_stair_user_occupation_id_bayes_rate_1': 0.053621666666666672,
 'context_page_stair_user_occupation_stair_bayes_rate_1': 0.053612,
 'context_page_stair_user_star_level_bayes_rate_2': 0.053624999999999999,
 'gender_age': 0.053614666666666665,
 'hour_bayes_rate_1': 0.053428333333333335,
 'hour_bayes_rate_2': 0.053434333333333334,
 'hour_gender_filled_bayes_rate_2': 0.053426000000000001,
 'hour_item_collected_level_bayes_rate_2': 0.053470333333333335,
 'hour_item_id_bayes_rate_2': 0.053445333333333324,
 'hour_item_price_stair_bayes_rate_2': 0.053434999999999989,
 'hour_item_pv_level_bayes_rate_2': 0.053456666666666659,
 'hour_item_pv_stair_bayes_rate_1': 0.053428333333333335,
 'hour_shop_id_bayes_rate_1': 0.053383333333333338,
 'hour_user_gender_id_bayes_rate_2': 0.053431666666666676,
 'hour_user_id_bayes_rate_2': 0.053580333333333334,
 'item_avg_age': 0.053598333333333338,
 'item_brand_collect_category_percent': 0.053589666666666667,
 'item_brand_id_isnew': 0.053609666666666667,
 'item_brand_id_user_gender_id_bayes_rate_2': 0.053256333333333329,
 'item_brand_id_user_gender_id_percent': 0.053581333333333335,
 'item_brand_id_user_occupation_stair_bayes_rate_1': 0.053233666666666672,
 'item_city_id_bayes_rate_2': 0.053520666666666668,
 'item_city_id_gender_filled_bayes_rate_1': 0.053525333333333334,
 'item_city_id_shop_review_num_level_bayes_rate_1': 0.053471999999999999,
 'item_city_id_shop_review_num_level_bayes_rate_2': 0.053474000000000001,
 'item_city_id_user_occupation_stair_bayes_rate_1': 0.053548999999999992,
 'item_city_price_category_percent': 0.053567333333333328,
 'item_city_pv_category_rank': 0.053582333333333336,
 'item_cnt_day': 0.053614000000000002,
 'item_cnt_dayavg_total': 0.053605666666666663,
 'item_collected_level_user_age_level_bayes_rate_2': 0.05362533333333333,
 'item_collected_level_user_id_bayes_rate_1': 0.053564999999999995,
 'item_collected_level_user_occupation_stair_bayes_rate_1': 0.053620000000000001,
 'item_id_gender_filled_bayes_rate_1': 0.052887999999999998,
 'item_id_shop_id_bayes_rate_1': 0.052852666666666659,
 'item_id_shop_review_num_level_bayes_rate_1': 0.052875999999999999,
 'item_id_user_gender_id_percent': 0.05357933333333334,
 'item_id_user_id_bayes_rate_1': 0.053580333333333341,
 'item_price_category_percent': 0.053516666666666664,
 'item_price_level_shop_star_level_bayes_rate_2': 0.053610333333333336,
 'item_price_level_user_id_bayes_rate_2': 0.053542333333333331,
 'item_price_level_user_occupation_id_bayes_rate_1': 0.053612333333333338,
 'item_price_level_user_star_level_bayes_rate_2': 0.053620666666666671,
 'item_price_stair_gender_filled_bayes_rate_1': 0.053602666666666667,
 'item_price_stair_shop_star_level_bayes_rate_1': 0.053617999999999999,
 'item_price_stair_shop_star_level_bayes_rate_2': 0.053616333333333328,
 'item_price_stair_user_age_level_bayes_rate_1': 0.053601000000000003,
 'item_price_stair_user_gender_id_bayes_rate_1': 0.053608666666666666,
 'item_price_stair_user_star_level_bayes_rate_1': 0.053626333333333331,
 'item_pv_level_shop_id_bayes_rate_1': 0.052949999999999997,
 'item_pv_level_shop_review_num_level_bayes_rate_1': 0.053615999999999997,
 'item_pv_level_shop_star_level_bayes_rate_2': 0.053616999999999998,
 'item_pv_level_user_age_stair_bayes_rate_2': 0.053623333333333335,
 'item_pv_level_user_id_bayes_rate_2': 0.053568999999999999,
 'item_pv_level_user_star_level_bayes_rate_2': 0.053629666666666666,
 'item_pv_stair_bayes_rate_1': 0.053606000000000008,
 'item_pv_stair_bayes_rate_2': 0.053606000000000008,
 'item_pv_stair_gender_filled_bayes_rate_1': 0.053605999999999994,
 'item_pv_stair_shop_review_num_level_bayes_rate_1': 0.053608333333333334,
 'item_pv_stair_user_occupation_id_bayes_rate_2': 0.053608999999999997,
 'item_pv_stair_user_occupation_stair_bayes_rate_1': 0.053621333333333333,
 'item_pv_stair_user_star_level_bayes_rate_1': 0.053615999999999997,
 'item_sales_div_pv': 0.05361966666666667,
 'item_sales_div_pv_percent': 0.053614000000000002,
 'item_sales_level_shop_review_num_level_bayes_rate_2': 0.053663999999999996,
 'item_sales_level_user_id_bayes_rate_2': 0.05357866666666667,
 'item_sales_level_user_occupation_stair_bayes_rate_2': 0.053606999999999995,
 'item_sales_stair_bayes_rate_2': 0.053616333333333328,
 'item_sales_stair_shop_star_level_bayes_rate_1': 0.053637000000000011,
 'item_today_query': 0.053602666666666667,
 'property_1': 0.053585333333333339,
 'property_21': 0.053606999999999995,
 'property_24': 0.053589333333333329,
 'property_26': 0.053575000000000005,
 'property_31': 0.053578999999999995,
 'property_33': 0.053583333333333337,
 'property_9': 0.053605666666666663,
 'property_count': 0.053560666666666666,
 'query_category_8': 0.053620000000000001,
 'query_categroy_count': 0.053618666666666669,
 'query_col_0': 0.053594666666666672,
 'query_col_10': 0.053609666666666667,
 'query_col_11': 0.053609666666666667,
 'sale_price': 0.053613999999999995,
 'shop_avg_star': 0.053588999999999998,
 'shop_current_hour_query': 0.053484666666666673,
 'shop_id_isnew': 0.053609666666666667,
 'shop_id_user_age_stair_bayes_rate_1': 0.05303133333333334,
 'shop_id_user_occupation_stair_bayes_rate_1': 0.053002666666666663,
 'shop_review_num_level_gender_filled_bayes_rate_1': 0.053609333333333335,
 'shop_review_num_level_gender_filled_bayes_rate_2': 0.053609666666666667,
 'shop_review_num_level_user_id_bayes_rate_1': 0.053575333333333329,
 'shop_review_num_level_user_star_stair_bayes_rate_2': 0.053611666666666669,
 'shop_star_level_gender_filled_bayes_rate_1': 0.053607666666666665,
 'shop_star_level_user_age_level_bayes_rate_2': 0.053620000000000001,
 'shop_star_level_user_id_bayes_rate_2': 0.053568333333333329,
 'shop_star_level_user_star_level_bayes_rate_2': 0.053622666666666659,
 'shop_star_level_user_star_stair_bayes_rate_1': 0.053612666666666663,
 'shop_today_query_percent': 0.05347933333333333,
 'time_slice_item_city_id_bayes_rate_1': 0.053473333333333338,
 'time_slice_item_pv_level_bayes_rate_2': 0.053461666666666664,
 'time_slice_shop_id_bayes_rate_2': 0.053473666666666669,
 'time_slice_user_age_stair_bayes_rate_2': 0.053433333333333333,
 'time_slice_user_gender_id_bayes_rate_1': 0.053422333333333329,
 'time_slice_user_gender_id_bayes_rate_2': 0.053416999999999999,
 'time_slice_user_occupation_id_bayes_rate_2': 0.053442333333333335,
 'time_slice_user_occupation_stair_bayes_rate_2': 0.053433666666666664,
 'time_slice_user_star_level_bayes_rate_2': 0.053473666666666669,
 'user_cnt_day': 0.053599666666666664,
 'user_isnew': 0.053584333333333324,
 'user_occupation_stair_bayes_rate_1': 0.053609666666666667,
 'user_query_collect_rank': 0.053270666666666661,
 'user_query_delivery_rank': 0.053198666666666672,
 'user_query_max_score10': 0.053545333333333334,
 'user_query_max_score2_rank': 0.053419666666666671,
 'user_query_max_score4_percent': 0.053078333333333338,
 'user_query_max_score5_rank': 0.053408000000000004,
 'user_query_max_score6_rank': 0.053318999999999998,
 'user_query_maxsales_div_pv': 0.053553666666666666,
 'user_query_maxshopstar': 0.053490333333333327,
 'user_query_price_percent': 0.053314,
 'user_query_pv_percent': 0.053360333333333336,
 'user_query_queryitem_c_similarity_rank2': 0.053164333333333334,
 'user_query_review_percent': 0.05338233333333333,
 'user_star_level_bayes_rate_2': 0.05362533333333333,
 'user_star_stair_bayes_rate_2': 0.053606999999999995,
 'user_today_query': 0.053326333333333337,
 'user_today_query_percent': 0.053330333333333334}

In [2]:
len(feature_performance)

860

In [4]:
good_features = []
baseline = 0.0536056666667

for i in sorted(feature_performance.items(), key=lambda x:x[1], reverse=False):
    if i[1] < baseline:
        good_features.append(i[0])
len(good_features)

460

In [5]:
good_features

['is_trade',
 'user_lastquery_timedelta',
 'user_lastquery',
 'item_price_level_shop_id_bayes_rate_2',
 'item_price_level_shop_id_bayes_rate_1',
 'item_price_stair_shop_id_bayes_rate_1',
 'item_id_bayes_rate_1',
 'item_price_stair_shop_id_bayes_rate_2',
 'item_id_shop_id_bayes_rate_1',
 'item_id_bayes_rate_2',
 'item_id_shop_id_bayes_rate_2',
 'item_id_shop_star_level_bayes_rate_1',
 'item_city_id_shop_id_bayes_rate_2',
 'item_id_shop_review_num_level_bayes_rate_2',
 'item_id_shop_star_level_bayes_rate_2',
 'item_sales_stair_shop_id_bayes_rate_2',
 'item_city_id_shop_id_bayes_rate_1',
 'item_sales_stair_shop_id_bayes_rate_1',
 'shop_id_bayes_rate_2',
 'item_id_shop_review_num_level_bayes_rate_1',
 'item_pv_stair_shop_id_bayes_rate_2',
 'item_brand_id_shop_id_bayes_rate_1',
 'item_pv_stair_shop_id_bayes_rate_1',
 'item_brand_id_shop_id_bayes_rate_2',
 'item_id_gender_filled_bayes_rate_1',
 'item_id_gender_filled_bayes_rate_2',
 'shop_id_gender_filled_bayes_rate_1',
 'shop_id_gender_fill

In [28]:
sorted(feature_performance.items(), key=lambda x:x[1], reverse=False)

[('is_trade', 3.0000000000000001e-06),
 ('user_lastquery_timedelta', 0.052493333333333336),
 ('user_lastquery', 0.052693666666666666),
 ('item_price_level_shop_id_bayes_rate_2', 0.052836333333333339),
 ('item_price_level_shop_id_bayes_rate_1', 0.052836666666666664),
 ('item_price_stair_shop_id_bayes_rate_1', 0.052850666666666664),
 ('item_price_stair_shop_id_bayes_rate_2', 0.052852666666666659),
 ('item_id_bayes_rate_1', 0.052852666666666659),
 ('item_id_shop_id_bayes_rate_2', 0.052853333333333335),
 ('item_id_bayes_rate_2', 0.052853333333333335),
 ('item_id_shop_star_level_bayes_rate_1', 0.052866000000000003),
 ('item_city_id_shop_id_bayes_rate_2', 0.052867999999999998),
 ('item_id_shop_review_num_level_bayes_rate_2', 0.052867999999999998),
 ('item_id_shop_star_level_bayes_rate_2', 0.05286833333333333),
 ('item_sales_stair_shop_id_bayes_rate_2', 0.052868666666666668),
 ('item_city_id_shop_id_bayes_rate_1', 0.052870333333333332),
 ('item_sales_stair_shop_id_bayes_rate_1', 0.05287233333